In [1]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d msambare/fer2013

Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
 98% 59.0M/60.3M [00:00<00:00, 103MB/s] 
100% 60.3M/60.3M [00:00<00:00, 89.3MB/s]


In [3]:
!unzip fer2013.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/sad/Training_65242339.jpg  
  inflating: train/sad/Training_65267116.jpg  
  inflating: train/sad/Training_65275626.jpg  
  inflating: train/sad/Training_6529266.jpg  
  inflating: train/sad/Training_65329617.jpg  
  inflating: train/sad/Training_65338712.jpg  
  inflating: train/sad/Training_65338797.jpg  
  inflating: train/sad/Training_65387162.jpg  
  inflating: train/sad/Training_65404494.jpg  
  inflating: train/sad/Training_65426218.jpg  
  inflating: train/sad/Training_65430136.jpg  
  inflating: train/sad/Training_65437377.jpg  
  inflating: train/sad/Training_6545735.jpg  
  inflating: train/sad/Training_65463385.jpg  
  inflating: train/sad/Training_65473985.jpg  
  inflating: train/sad/Training_65502829.jpg  
  inflating: train/sad/Training_65505359.jpg  
  inflating: train/sad/Training_65508578.jpg  
  inflating: train/sad/Training_65516023.jpg  
  inflating: train/sad/Training_65524027.jpg  
  inflating

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
import cv2 as cv
import os

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
class EmotionRecognitionDatset(Dataset):
    def __init__(self, data_path):
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((48, 48)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485], std=[0.229])
        ])
        self.data = {}
        for label in os.listdir(data_path):
            for image in os.listdir(os.path.join(data_path, label)):
                self.data[os.path.join(data_path, label, image)] = label
        self.data = pd.DataFrame(list(self.data.items()), columns=['image', 'label'])
        labels_encoded = pd.get_dummies(self.data['label'])
        self.data = self.data.drop(['label'], axis=1).join(labels_encoded)
        self.n_labels = len(os.listdir(data_path))


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = cv.imread(self.data.iloc[idx]['image'])
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image = self.transforms(image)
        label = self.data.iloc[idx,1:]
        label = torch.tensor(label, dtype=torch.float32)
        return image, label

In [7]:
dataset = EmotionRecognitionDatset('./train')
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [8]:
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s]


In [9]:
model.fc = torch.nn.Linear(model.fc.in_features, dataset.n_labels)

In [10]:
model = model.to(device)

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
epochs = 10

In [12]:
for epoch in range(epochs):
    for i, (images, labels) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f"Epoch: {epoch+1}/{epochs}, iter: {i}/{len(dataloader)}, loss: {loss.item()}")

Epoch: 1/10, iter: 0/898, loss: 2.033730983734131
Epoch: 1/10, iter: 10/898, loss: 2.115609884262085
Epoch: 1/10, iter: 20/898, loss: 2.058950901031494
Epoch: 1/10, iter: 30/898, loss: 1.9339351654052734
Epoch: 1/10, iter: 40/898, loss: 2.011895179748535
Epoch: 1/10, iter: 50/898, loss: 2.076554775238037
Epoch: 1/10, iter: 60/898, loss: 1.9497363567352295
Epoch: 1/10, iter: 70/898, loss: 1.9411245584487915
Epoch: 1/10, iter: 80/898, loss: 1.8502922058105469
Epoch: 1/10, iter: 90/898, loss: 1.8652360439300537
Epoch: 1/10, iter: 100/898, loss: 1.8155158758163452
Epoch: 1/10, iter: 110/898, loss: 1.6308081150054932
Epoch: 1/10, iter: 120/898, loss: 1.773113489151001
Epoch: 1/10, iter: 130/898, loss: 1.790402889251709
Epoch: 1/10, iter: 140/898, loss: 1.867081880569458
Epoch: 1/10, iter: 150/898, loss: 1.8430087566375732
Epoch: 1/10, iter: 160/898, loss: 1.747308373451233
Epoch: 1/10, iter: 170/898, loss: 1.7877752780914307
Epoch: 1/10, iter: 180/898, loss: 1.9142415523529053
Epoch: 1/10, 

In [13]:
torch.save(model.state_dict(), 'emotion_recognition_model.pth')

In [14]:
for epoch in range(epochs):
    for i, (images, labels) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f"Epoch: {epoch+1}/{epochs}, iter: {i}/{len(dataloader)}, loss: {loss.item()}")

Epoch: 1/10, iter: 0/898, loss: 0.06355830281972885
Epoch: 1/10, iter: 10/898, loss: 0.25067293643951416
Epoch: 1/10, iter: 20/898, loss: 0.2223944365978241
Epoch: 1/10, iter: 30/898, loss: 0.12081048637628555
Epoch: 1/10, iter: 40/898, loss: 0.0780993327498436
Epoch: 1/10, iter: 50/898, loss: 0.1964685022830963
Epoch: 1/10, iter: 60/898, loss: 0.04582418501377106
Epoch: 1/10, iter: 70/898, loss: 0.11747067421674728
Epoch: 1/10, iter: 80/898, loss: 0.08577500283718109
Epoch: 1/10, iter: 90/898, loss: 0.05956026539206505
Epoch: 1/10, iter: 100/898, loss: 0.08802279829978943
Epoch: 1/10, iter: 110/898, loss: 0.1093718484044075
Epoch: 1/10, iter: 120/898, loss: 0.104140505194664
Epoch: 1/10, iter: 130/898, loss: 0.11283396929502487
Epoch: 1/10, iter: 140/898, loss: 0.06314946711063385
Epoch: 1/10, iter: 150/898, loss: 0.11202767491340637
Epoch: 1/10, iter: 160/898, loss: 0.11265013366937637
Epoch: 1/10, iter: 170/898, loss: 0.025954745709896088
Epoch: 1/10, iter: 180/898, loss: 0.08280695

In [15]:
torch.save(model.state_dict(), 'emotion_recognition_model2.pth')